In [10]:
#Importações
import numpy as np
import pandas as pd
from sklearn.datasets import load_iris
from sklearn.datasets import load_boston

iris = load_iris()
table_iris = pd.DataFrame(iris.data)
table_iris.columns = iris.feature_names
table_iris['Class'] = iris.target

boston = load_boston()
table_boston = pd.DataFrame(boston.data)
table_boston.columns = boston.feature_names
table_boston['Value'] = boston.target

In [11]:
#EXIBIÇÃO: iris
table_iris.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),Class
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0


In [12]:
#EXIBIÇÃO: boston
table_boston.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,Value
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,36.2


In [ ]:
#discretiza os preços em 4 Valores
table_boston = table_boston.sort_values(by=['Value'])
table_boston = table_boston.reset_index(drop=True)
for i in range(126):
    table_boston.iloc[i]['Value'] = 4

for i in range(126, 253):
    table_boston.iloc[i]['Value'] = 3

for i in range(253, 380):
    table_boston.iloc[i]['Value'] = 2

for i in range(380, 506):
    table_boston.iloc[i]['Value'] = 1
    
table_boston['Value'] = pd.to_numeric(table_boston['Value'])

C:\Users\David\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


In [ ]:
#EXIBIÇÃO: boston
table_boston.head(10)